In [18]:
import tensorflow as tf
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt

In [33]:
train_path = './data/train_images_split/train'
val_path = './data/train_images_split/val'
test_path = './data/test_images'

img_size = 224

In [34]:
X_train = []
x = 0
for folder in os.listdir(train_path):
    
    sub_path = train_path + '/' + folder
    
    for img in os.listdir(sub_path):
        image_path = sub_path + '/' + img
        img_arr = cv2.imread(image_path)
        img_arr = cv2.resize(img_arr, (img_size, img_size))
        X_train.append(img_arr)
        x += 1
        print(x, end='\r')

In [35]:
X_test = []
x = 0
for img in os.listdir(test_path):
    
    image_path = test_path + '/' + img
    img_arr = cv2.imread(image_path)
    img_arr = cv2.resize(img_arr, (img_size, img_size))
    X_test.append(img_arr)
    x += 1
    print(x, end='\r')

In [36]:
X_val = []
x = 0
for folder in os.listdir(val_path):
    sub_path = val_path + '/' + folder
    
    for img in os.listdir(sub_path):
        image_path = sub_path + '/' + img
        img_arr = cv2.imread(image_path)
        img_arr = cv2.resize(img_arr, (img_size, img_size))
        X_val.append(img_arr)
        x += 1
        print(x, end = '\r')

In [45]:
X_train = np.array(X_train)
X_test = np.array(X_test)
X_val = np.array(X_val)

X_train = X_train / 255
X_test = X_test / 255
X_val = X_val / 255

print(X_train)

MemoryError: Unable to allocate 9.33 GiB for an array with shape (8323, 224, 224, 3) and data type float64

In [38]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
print(train_datagen)

In [39]:
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (img_size,img_size),
                                                 batch_size = 32,
                                                 class_mode = 'sparse')
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (img_size,img_size),
                                            batch_size = 32,
                                            class_mode = 'sparse')
val_set = val_datagen.flow_from_directory(val_path,
                                            target_size = (img_size,img_size),
                                            batch_size = 32,
                                            class_mode = 'sparse')

Found 8323 images belonging to 10 classes.
Found 0 images belonging to 0 classes.
Found 2084 images belonging to 10 classes.


In [40]:
y_train=training_set.classes
y_test=test_set.classes
y_val=val_set.classes

training_set.class_indices

{'bacterial_leaf_blight': 0,
 'bacterial_leaf_streak': 1,
 'bacterial_panicle_blight': 2,
 'blast': 3,
 'brown_spot': 4,
 'dead_heart': 5,
 'downy_mildew': 6,
 'hispa': 7,
 'normal': 8,
 'tungro': 9}

In [41]:
y_train.shape,y_test.shape,y_val.shape

((8323,), (0,), (2084,))

In [42]:
X_train.shape[1]

224

In [43]:
vgg = tf.keras.applications.VGG19(input_shape = (img_size, img_size, 3), weights = 'imagenet', include_top = False)

In [47]:
# Make all the layers in the pre-trained model non-trainable
for layer in vgg.layers:
    layer.trainable = False

NameError: name 'vgg' is not defined

In [44]:
x =  tf.keras.layers.Flatten()(vgg.output)
#adding output layer.Softmax classifier is used as it is multi-class classification
prediction =  tf.keras.layers.Dense(10, activation='softmax')(x)

model = tf.keras.models.Model(inputs=vgg.input, outputs=prediction)
# view the structure of the model
model.summary()

NameError: name 'vgg' is not defined

In [ ]:
model.compile(
  loss='sparse_categorical_crossentropy',
  optimizer="adam",
  metrics=['accuracy']
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=5)
#Early stopping to avoid overfitting of model

In [ ]:
# fit the model
history = model.fit(
  X_train,
  y_train,
  validation_data=(X_val,y_val),
  epochs=10,
  callbacks=[early_stop],
  batch_size=32,shuffle=True)

In [ ]:

acc      = history.history[     'accuracy' ]
val_acc  = history.history[ 'val_accuracy' ]
loss     = history.history[    'loss' ]
val_loss = history.history['val_loss' ]

epochs   = range(len(acc)) 

plt.plot  ( epochs,     acc, label='Training')
plt.plot  ( epochs, val_acc, label='Validation')
plt.title ('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot  ( epochs,     loss, label='Training')
plt.plot  ( epochs, val_loss, label='Validation')
plt.legend()
plt.title ('Training and validation loss')

In [ ]:
# save model
saved_model_path = "./paddy_model.h5"
model.save(saved_model_path)

In [ ]:
# save sebagai tfjs
!tensorflowjs_converter --input_format=keras {saved_model_path} 

In [ ]:
# EXERCISE: Use the tf.saved_model API to save your model in the SavedModel format. 
export_dir = 'saved_model_tflite'

# YOUR CODE HERE
tf.saved_model.save(model,export_dir=export_dir)

In [ ]:
#@title Select mode of optimization
mode = "Speed" #@param ["Default", "Storage", "Speed"]

if mode == 'Storage':
    optimization = tf.lite.Optimize.OPTIMIZE_FOR_SIZE
elif mode == 'Speed':
    optimization = tf.lite.Optimize.OPTIMIZE_FOR_LATENCY
else:
    optimization = tf.lite.Optimize.DEFAULT

In [ ]:
# Use the TFLiteConverter SavedModel API to initialize the converter
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)

# Set the optimzations
converter.optimizations = [optimization]

# Invoke the converter to finally generate the TFLite model
tflite_model =  converter.convert()

In [ ]:
tflite_model_file = pathlib.Path('./model.tflite')
tflite_model_file.write_bytes(tflite_model)